# Yandex.Afisha analysis
We have files with visits and orders from June 2017 to May 2018 along with some statistics on marketing expenses. The idea is to find out how people use the product, focussing on when they bring in money and how much. Specifically, we have three tables:

* Visits, of which each row contains timestamps for the start and end of a visit, an ID for the user, what type of device they were on (two categories, don't get too excited), and what kind of source they came from
* Orders, with one observation for each order, containing a user ID, a timestamp, and amount of revenue
* Costs, the marketing expenses table, with a column each for source, date, and expenses

Some of these files are pretty big, so let's bring them in carefully.

## Loading the tables and making fun of their formatting

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

visits = pd.read_csv("datasets/visits_log_us.csv", nrows=1000)
visits.info(memory_usage="deep")

Yes, these column names are awful. Does it really matter when everything will be in quotes anyway, no, but we'll fix it in a moment.

In [ ]:
display(visits.head(10))

So there are some data types we'll want to change if we're going to analyse the whole 24 MB file. Device should be categorical and the timestamps should be timestamps. This is also a good time to show the "two categories" thing:

In [ ]:
visits["Device"].value_counts()

No creepy captures of screen resolution and CPU specs for us. Now let's import the table with data types that won't make everything slow.

In [ ]:
visits = pd.read_csv('datasets/visits_log_us.csv', dtype={"Device": "category"}, parse_dates=["Start Ts", "End Ts"])
visits = visits.rename(columns={"Device": "device", "End Ts": "end_ts", "Source Id": "source_id", "Start Ts": "start_ts", "Uid": "uid"})

In [ ]:
visits.info(memory_usage='deep')

In [ ]:
visits.duplicated(subset=["uid", "start_ts"]).sum()

In [ ]:
visits[visits.duplicated(subset=["uid", "start_ts"], keep=False)]

The simple explanation is that someone left and came back to the site within one minute. I know I've done that sort of thing, so there's no good reason to think this is a mistake.

In [ ]:
orders = pd.read_csv("datasets/orders_log_us.csv", nrows=1000)
orders.info()

That's right, more annoying column names.

In [ ]:
orders.head(10)

In [ ]:
orders = pd.read_csv('datasets/orders_log_us.csv', parse_dates=["Buy Ts"])
orders = orders.rename(columns={"Buy Ts": "buy_ts", "Revenue": "revenue", "Uid": "uid"})
orders.info()

In [ ]:
orders.duplicated(subset=["uid", "buy_ts"]).sum()

In [ ]:
costs = pd.read_csv("datasets/costs_us.csv")
costs.info()

In [ ]:
costs.head(10)

In [ ]:
costs["source_id"].value_counts()

In [ ]:
costs.duplicated(subset=["source_id", "dt"]).sum()

It'd be nice to know within the database what these sources actually are. There's no "sources" table. In a real situation, someone around would know (hopefully), so it wouldn't be a big deal, but talking about "source 3" will be weird. Anyway, we've got a column to convert.

In [ ]:
costs["dt"] = pd.to_datetime(costs["dt"])

## Let's actually calculate something now
### Product
Time for some basic usage statistics. Specifically, number of users by day, week, and month; sessions per day; and length of sessions.

#### Users by lengths of time

In [ ]:
visits["day"] = visits["start_ts"].astype("datetime64[D]")
visits["week"] = visits["start_ts"].astype("datetime64[W]")
visits["month"] = visits["start_ts"].astype("datetime64[M]")
visits.head()

In [ ]:
grouped = visits.drop_duplicates(["uid", "month"])
fig = px.histogram(grouped, x="month", color="device")
fig.update_layout(title="Unique users plateaued from October to March", xaxis_title="Month", yaxis_title="Users", legend_traceorder="reversed", legend_title_text="Device")
item_labels = ["Touch", "Desktop"]
for i, label in enumerate(item_labels):
    fig.data[i].name = label
fig.show()

We have an encouraging rise over September and October, but then what happened to make the drop from March to April? Maybe nothing but a seasonal effect because we only have twelve months to work with, but maybe something there's something to be done about.

In [ ]:
grouped = visits.drop_duplicates(["uid", "week"])
fig = px.histogram(grouped, x="week", color="device", nbins=visits["week"].nunique()*2-2)
fig.update_layout(title="Unique users plateaued from October to March", xaxis_title="Week", yaxis_title="Users", legend_traceorder="reversed", legend_title_text="Device")
item_labels = ["Touch", "Desktop"]
for i, label in enumerate(item_labels):
    fig.data[i].name = label
fig.show()

By week we can see more clearly specifically when the rise was: there was a positive trend over August too, even though the month didn't have that many visits, and by mid-October it was finished.

In [ ]:
grouped = visits.drop_duplicates(["uid", "day"])
fig = px.histogram(grouped, x="day", nbins=visits["day"].nunique()*2-2)
fig.update_layout(title="November 24 was a big day", xaxis_title="Day", yaxis_title="Users")
fig.show()

We've got day of the week effects. Also, what happened on those big days? Let's get a nice list of which days they are.

In [ ]:
grouped["day"].value_counts()

The time around the end of March needs to be acknowledged too, probably the site was down for whatever reason. I doubt it's anything related to the lesser numbers of the last two months. But now we can ask specifically: what fun thing was going on on November 24th. Maybe someone will know, because this dataset alone can't tell us.

The days of the week are one group of 5 and one of 2, so probably weekdays and weekends, but let's actually see.

In [ ]:
grouped["day"].dt.day_of_week.value_counts()

Indeed, Saturday and Sunday are less popular.

Here are the averages for unique users:

In [ ]:
visits_d = visits[visits["device"] == "desktop"]
visits_t = visits[visits["device"] == "touch"]

users_per_time = pd.DataFrame(index=["Daily", "Weekly", "Monthly"], data={
    "Desktop": [visits_d.groupby("day").agg({"uid": "nunique"})["uid"].mean(), visits_d.groupby("week").agg({"uid": "nunique"})["uid"].mean(), visits_d.groupby("month").agg({"uid": "nunique"})["uid"].mean()],
    "Touch": [visits_t.groupby("day").agg({"uid": "nunique"})["uid"].mean(), visits_t.groupby("week").agg({"uid": "nunique"})["uid"].mean(), visits_t.groupby("month").agg({"uid": "nunique"})["uid"].mean()],
    "Total": [visits.groupby("day").agg({"uid": "nunique"})["uid"].mean(), visits.groupby("week").agg({"uid": "nunique"})["uid"].mean(), visits.groupby("month").agg({"uid": "nunique"})["uid"].mean()]
})
display(users_per_time)

#### How many sessions are there per day?

In [ ]:
visits_by_day = visits.groupby("day", as_index=False).agg({"start_ts": "count"})
visits_by_day_d = visits_d.groupby("day", as_index=False).agg({"start_ts": "count"})
visits_by_day_t = visits_t.groupby("day", as_index=False).agg({"start_ts": "count"})

#same thing with urge to group: print(visits.groupby("day").count().mean())
print("Daily average visits (desktop):", visits_by_day_d["start_ts"].mean())
print("Daily average visits (touch):", visits_by_day_t["start_ts"].mean())
print("Daily average visits (total):", visits_by_day["start_ts"].mean())

There you go. Something like unique visitors by week or month might be more analysable, but we just did that. And a single stat has its uses too.

After users and sessions, sessions per user seems like something to do. First off, has there been a trend?

In [ ]:
grouped = visits.groupby(["uid", "day"], as_index=False).agg({"start_ts": "count"})
daymeans = grouped.groupby("day", as_index=False).agg({"start_ts": "mean"})
daymedians = grouped.groupby("day", as_index=False).agg({"start_ts": "median"})

line = px.line(daymeans, x="day", y="start_ts")
line.update_layout(title="Average visits by a user by day didn't change much", xaxis_title="Day", yaxis_title="Average visits per user")
line.show()

Not really. Look at the scale of this graph, it's only 1 to 1.2, and even that is thanks to a couple of outliers, if you can even call them that. There *was* an upward trend throughout May, but looking at the rest of the year, it feels more like a recovery than anything.

And have there been any users making the mean misleading with their big numbers?

In [ ]:
usermeans = grouped.groupby("uid", as_index=False).agg({"start_ts": "mean"})
userhist = px.histogram(usermeans, x="start_ts")
userhist.update_layout(title="1-visit-per-day users absolutely dominate", xaxis_title="Average visits per day", yaxis_title="Users")
userhist.show()

In [ ]:
print("Average of users' average visits per day:", usermeans["start_ts"].mean())

Nope. Here's a corresponding histogram for that line chart of daily averages:

In [ ]:
hist = px.histogram(daymeans, x="start_ts")
hist.update_traces(xbins_size=0.01)
hist.update_layout(title="Visits per user on each day is generally 1.04 to 1.12", xaxis_title="Average visits per user", yaxis_title="Days")
hist.show()

The fact that bins a hundredth of a visit wide are appropriate drives home there isn't much variance. So since the distribution is a pretty typical bell curve and even our outliers aren't that far away, let's finally get the number, what's the mean of all this.

In [ ]:
print("Daily average of visits per user:", daymeans["start_ts"].mean())

As the chart by user shows, making more than one visit in a day, or especially more than two: it's weird. The mode is 1, and leaving it at that wouldn't be misleading.

#### What's the length of each session?
Because the timestamps are to minute resolution, we'll have it in minutes.

In [ ]:
visits["duration"] = visits["end_ts"] - visits["start_ts"]
visits["duration"] = (visits["duration"].dt.total_seconds() / 60).astype("int")
visits.head()

In [ ]:
fig = px.histogram(visits, x="duration")
fig.show()

That isn't that helpful. Let's limit it to two hours or less.

In [ ]:
fig = px.histogram(visits[visits["duration"] <= 120], x="duration", color="device")
fig.update_layout(title="Most visits are brief", xaxis_title="Minutes", yaxis_title="Visits", legend_title_text="Device")
item_labels = ["Touch", "Desktop"]
for i, label in enumerate(item_labels):
    fig.data[i].name = label
fig.show()

In [ ]:
from scipy import stats
nonn_duration = visits[visits["duration"] >= 0]["duration"]

print("Mean visit length:", nonn_duration.mean(), "minutes")
print("50% trimmed mean visit length:", stats.trim_mean(nonn_duration, 0.25), "minutes")
print("Median visit length:", nonn_duration.median(), "minutes")
print("Most common visit length:", nonn_duration.mode()[0], "minute")

Most visits are for a few minutes or even less. After that we get a nice, smooth curve with an apex at around 15 minutes, then a weird step at around half an hour. The middle 50% of visits have an average of 5 minutes and 47 seconds, but then the most common length is 1 minute. (And not just because that's the minimum; there is 0.)

Also, why are there negative values??

In [ ]:
display(visits[visits["start_ts"] > visits["end_ts"]])

They're in the hour of 3 AM... I thought maybe it's a summer time thing, but Russia got rid of it years before the dates in question. But it seems on that hour of March 25, 2018, *something* weird was going on with the site. There's a lot of "someone else should know" today, but sometimes that's what bare tables do, raise more questions.

#### Time for retention rates
And time to add more columns to the visits table!

In [ ]:
first_month = visits.groupby("uid")["start_ts"].min()
first_month.name = "first_month"
visits = visits.join(first_month, on="uid")
visits["first_month"] = visits["first_month"].astype("datetime64[M]")
visits.head()

In [ ]:
visits.pivot_table(index="first_month", columns="month", values="uid", aggfunc="nunique")

The retention rate is not great. Let's do the same thing in lifetime terms:

In [ ]:
visits["lt_month"] = visits["month"] - visits["first_month"]
visits["lt_month"] = visits["lt_month"] / np.timedelta64(1, "M")
visits["lt_month"] = visits["lt_month"].round().astype("int")
visits.head()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
retention_pivot = visits.pivot_table(index="first_month", columns="lt_month", values="uid", aggfunc="nunique")

plt.figure(figsize=(15, 8))
plt.title("A large fraction of users don't come back after the first month")
sns.heatmap(retention_pivot, annot=True, vmin=0, vmax=2500, fmt=".0f", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

In [ ]:
rpfrac = visits.pivot_table(index="first_month", columns="lt_month", values="uid", aggfunc="nunique")
rpfrac = rpfrac.divide(rpfrac.iloc[:, 0], axis=0)
plt.figure(figsize=(15, 8))
plt.title("A large fraction of users don't come back after the first month")
sns.heatmap(rpfrac, annot=True, vmin=0, vmax=0.1, fmt=".3f", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

After the initial big dropoff, the curve is much flatter for the first two cohorts. August is in the middle of the decline, and from September on things are pretty stable. Not *good*, with a half-life of two or three months, but stable. And even that isn't good with the said dropoff getting worse over time. The story applies to both types of device too. The tables just for desktop and touch respectively:

In [ ]:
visits_d = visits[visits["device"] == "desktop"]
visits_t = visits[visits["device"] == "touch"]
vdpivot = visits_d.pivot_table(index="first_month", columns="lt_month", values="uid", aggfunc="nunique")
vtpivot = visits_t.pivot_table(index="first_month", columns="lt_month", values="uid", aggfunc="nunique")

plt.figure(figsize=(15, 8))
plt.title("A large fraction of users don't come back after the first month")
sns.heatmap(vdpivot, annot=True, vmin=0, vmax=1728, fmt=".0f", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

In [ ]:
visits_d = visits[visits["device"] == "desktop"]
visits_t = visits[visits["device"] == "touch"]
vdpivot = visits_d.pivot_table(index="first_month", columns="lt_month", values="uid", aggfunc="nunique")
vtpivot = visits_t.pivot_table(index="first_month", columns="lt_month", values="uid", aggfunc="nunique")

plt.figure(figsize=(15, 8))
plt.title("A large fraction of users don't come back after the first month")
sns.heatmap(vtpivot, annot=True, vmin=0, vmax=600, fmt=".0f", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

### Sales
When (in lifetime terms) do people make their first order, how many orders is typical per unit time, and what are normal purchase sizes and lifetime values?

#### When do people first buy something?

In [ ]:
user_first_months = pd.DataFrame({"uid": visits["uid"], "first_month": visits["first_month"]})
user_first_months = user_first_months.drop_duplicates()

user_ltvs = orders.groupby("uid", as_index=False).agg({"revenue": "sum"})

user_first_orders = orders.groupby("uid", as_index=False).agg({"buy_ts": "min"})
user_first_orders["buy_ts"] = user_first_orders["buy_ts"].astype("datetime64[D]")
user_first_orders = user_first_orders.rename(columns={"buy_ts": "first_order_day"})

user_first_visits = visits.groupby("uid", as_index=False).agg({"start_ts": "min"})
user_first_visits["start_ts"] = user_first_visits["start_ts"].astype("datetime64[D]")
user_first_visits = user_first_visits.rename(columns={"start_ts": "first_day"})

user_info = user_first_months.merge(user_ltvs, on="uid", how="left")
user_info = user_info.fillna(value={"revenue": 0})
user_info = user_info.merge(user_first_orders, on="uid", how="left")
user_info = user_info.merge(user_first_visits, on="uid", how="left")
user_info["days_to_buy"] = ((user_info["first_order_day"] - user_info["first_day"]) / np.timedelta64(1, "D"))

display(user_info.head())

In [ ]:
display(user_info.groupby("first_month").agg({"days_to_buy": "mean"}))

The time to the first purchase has been getting shorter. But it could be because new people who haven't bought anything don't have a chance to appear in the table and contribute their high numbers. Let's have a look at the distribution for the first month....

In [ ]:
fig = px.histogram(user_info[user_info["first_month"] == "2017-06-01"], x="days_to_buy")
fig.update_layout(title="Most people who buy do it in their first few days", xaxis_title="Days to first purchase", yaxis_title="Users")
fig.show()

Buying in the first several days is common among those who do, then we've got a small but fairly steady stream of older users, making mean not the best function. In fact, there probably is no best function. Maybe mode. Either way, you should still visualise. And yes, the other months have the same thing going. Have a look at these eleven really similar charts if you want to see:

In [ ]:
px.histogram(user_info[user_info["first_month"] == "2017-07-01"], x="days_to_buy").show()
px.histogram(user_info[user_info["first_month"] == "2017-08-01"], x="days_to_buy").show()
px.histogram(user_info[user_info["first_month"] == "2017-09-01"], x="days_to_buy").show()
px.histogram(user_info[user_info["first_month"] == "2017-10-01"], x="days_to_buy").show()
px.histogram(user_info[user_info["first_month"] == "2017-11-01"], x="days_to_buy").show()
px.histogram(user_info[user_info["first_month"] == "2017-12-01"], x="days_to_buy").show()
px.histogram(user_info[user_info["first_month"] == "2018-01-01"], x="days_to_buy").show()
px.histogram(user_info[user_info["first_month"] == "2018-02-01"], x="days_to_buy").show()
px.histogram(user_info[user_info["first_month"] == "2018-03-01"], x="days_to_buy").show()
px.histogram(user_info[user_info["first_month"] == "2018-04-01"], x="days_to_buy").show()
px.histogram(user_info[user_info["first_month"] == "2018-05-01"], x="days_to_buy").show()

So let's not separate them by month, and let's also zoom in on those who took less than 30 days.

In [ ]:
fig = px.histogram(user_info[user_info["days_to_buy"] <= 30], x="days_to_buy")
fig.update_layout(title="Most who convert do it on their first day", xaxis_title="Days to first purchase", yaxis_title="Users")
fig.show()

Converting after you've been around for a while isn't the common thing.

So all the cohorts are pretty much the same in this regard. What about the sources, then? Some users will of course have more than one source:

In [ ]:
print("Users:",visits.drop_duplicates(subset=["uid", "source_id"])["uid"].nunique())
print("User-source combinations:",visits.drop_duplicates(subset=["uid", "source_id"])["uid"].count())

But taking the source of someone's earliest visit seems like a sensible way to determine the "actual source" of them.

In [ ]:
user_sources = visits.sort_values("start_ts", ascending=True).drop_duplicates(subset="uid")
user_sources = pd.DataFrame({"uid": user_sources["uid"], "first_source": user_sources["source_id"]})
user_info = user_info.merge(user_sources, on="uid", how="left")
display(user_info.head())

In [ ]:
fig = px.histogram(user_info[user_info["first_source"] == 1], x="days_to_buy")
fig.update_layout(title="Source 1", xaxis_title="Days to first purchase", yaxis_title="Users")
fig.show()

fig = px.histogram(user_info[user_info["first_source"] == 2], x="days_to_buy")
fig.update_layout(title="Source 2", xaxis_title="Days to first purchase", yaxis_title="Users")
fig.show()

fig = px.histogram(user_info[user_info["first_source"] == 3], x="days_to_buy")
fig.update_layout(title="Source 3", xaxis_title="Days to first purchase", yaxis_title="Users")
fig.show()

fig = px.histogram(user_info[user_info["first_source"] == 4], x="days_to_buy")
fig.update_layout(title="Source 4", xaxis_title="Days to first purchase", yaxis_title="Users")
fig.show()

fig = px.histogram(user_info[user_info["first_source"] == 5], x="days_to_buy")
fig.update_layout(title="Source 5", xaxis_title="Days to first purchase", yaxis_title="Users")
fig.show()

fig = px.histogram(user_info[user_info["first_source"] == 9], x="days_to_buy")
fig.update_layout(title="Source 9", xaxis_title="Days to first purchase", yaxis_title="Users")
fig.show()

fig = px.histogram(user_info[user_info["first_source"] == 10], x="days_to_buy")
fig.update_layout(title="Source 10", xaxis_title="Days to first purchase", yaxis_title="Users")
fig.show()

Sources 2 and 9 have more "hopeful" curves, with things not dropping off to near zero after the first couple weeks. Still, the numbers are seriously low, and the first few days being dominant is the case for all of them. You could also say this is all a good thing in that people who make a purchase do it quickly after they come. A better way to do this might be lifetime value or ROI by source, and we'll get to that.

#### Orders per unit time
How many orders per day or whatever other unit? We already have charts of this to compare between months and so on, but as for averages:

In [ ]:
print("Orders per day:",(orders["buy_ts"].count() / 365).round().astype("int"))
print("Orders per week:",(orders["buy_ts"].count() / 52.143).round().astype("int"))
print("Orders per month:",(orders["buy_ts"].count() / 12).round().astype("int"))

What about per person? Is it mostly lots of people making one or two orders or a small number of people making a lot? And has that been changing?

In [ ]:
orders["day"] = orders["buy_ts"].astype("datetime64[D]")
orders["week"] = orders["buy_ts"].astype("datetime64[W]")
orders["month"] = orders["buy_ts"].astype("datetime64[M]")

users_orders_per_day = orders.groupby(["uid", "day"], as_index=False).agg({"buy_ts": "count"})
w_grouped_d = users_orders_per_day.groupby("day", as_index=False).agg({"buy_ts": "mean"})
fig = px.line(w_grouped_d, x="day", y="buy_ts")
fig.update_layout(title="Purchases per user by day didn't change over time", xaxis_title="Day", yaxis_title="Mean purchases by a user")
fig.update_yaxes(range=[1, 1.25])
fig.show()

users_orders_per_week = orders.groupby(["uid", "week"], as_index=False).agg({"buy_ts": "count"})
w_grouped_w = users_orders_per_week.groupby("week", as_index=False).agg({"buy_ts": "mean"})

users_orders_per_month = orders.groupby(["uid", "month"], as_index=False).agg({"buy_ts": "count"})
w_grouped_m = users_orders_per_month.groupby("month", as_index=False).agg({"buy_ts": "mean"})

print("Mean number of purchases by a user on a day:",users_orders_per_day["buy_ts"].mean())
print("Mean number of purchases per user by day:",w_grouped_d["buy_ts"].mean())
print("Mean number of purchases by a user in a week:",users_orders_per_week["buy_ts"].mean())
print("Mean number of purchases per user by week:",w_grouped_w["buy_ts"].mean())
print("Mean number of purchases by a user in a month:",users_orders_per_month["buy_ts"].mean())
print("Mean number of purchases per user by month:",w_grouped_m["buy_ts"].mean())

Note the range of the chart: 1 to 1.25. Even then there's no long-term trend to find.

Whatever time period you use, the mean purchases by a user in it is pretty close to 1. It's related to the above numbers: most purchasers pop in to do it once and then don't think about the place again, or at least buy again. We should try to get better at converting people into repeat customers.

To show that lots of people making a small number of purchases is what's dominating the numbers, here's an actual chart of how many purchases (not just their first one) people make in a month. Numbers of purchases in a month with fewer than 100 cases are excluded (as is 0 purchases, like the previous chart).

In [ ]:
data = users_orders_per_month.groupby("buy_ts", as_index=False).agg({"uid": "count"})
data = data[data["uid"] >= 100]
fig = px.bar(data, x="buy_ts", y="uid")
fig.update_layout(title="Making multiple purchases in a month is rare", xaxis_title="Purchases", yaxis_title="Users in a month")
fig.show()

How about by cohort and lifetime month?

In [ ]:
users_orders_per_month = orders.groupby(["uid", "month"], as_index=False).agg({"buy_ts": "count"})
users_orders_per_month = users_orders_per_month.merge(user_sources, how="left", on="uid")
users_orders_per_month = users_orders_per_month.merge(user_first_months, how="left", on="uid")

users_orders_per_month["lt_month"] = users_orders_per_month["month"] - users_orders_per_month["first_month"]
users_orders_per_month["lt_month"] = users_orders_per_month["lt_month"] / np.timedelta64(1, "M")
users_orders_per_month["lt_month"] = users_orders_per_month["lt_month"].round().astype("int")

uopm_pt = users_orders_per_month.pivot_table(index="first_month", columns="lt_month", values="buy_ts", aggfunc="mean")
plt.figure(figsize=(15, 8))
plt.title("In any given month, people of the June 2017 cohort made the most purchases in months they did")
sns.heatmap(uopm_pt, annot=True, fmt=".2f", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"], cmap="YlGn", vmin=1, vmax=2.5)
plt.xlabel("Lifetime month")
plt.ylabel("First month")

There aren't any huge differences here, other than June 2017 being amazing. But if we exclude the weird cases, that is, a user making more than 4 purchases in a month....

In [ ]:
uopm_normal = users_orders_per_month[users_orders_per_month["buy_ts"] <= 4]

uopmn_pt = uopm_normal.pivot_table(index="first_month", columns="lt_month", values="buy_ts", aggfunc="mean")
plt.figure(figsize=(15, 8))
plt.title("The June 2017 cohort is the one that likes buying multiple times a month the most")
sns.heatmap(uopmn_pt, annot=True, fmt=".2f", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"], cmap="YlGn", vmin=1, vmax=1.333)
plt.xlabel("Lifetime month")
plt.ylabel("First month")

The difference isn't so dramatic anymore, but still: counting only those who did buy in a month, people of the June 2017 cohort usually made the most purchases.

#### The average purchase size
The mean purchase size is this:

In [ ]:
orders["revenue"].mean()

Let's just call it 5. Or should we...?

In [ ]:
fig = px.histogram(orders, x="revenue")
fig.show()

In [ ]:
fig = px.histogram(orders[orders["revenue"] <= 50], x="revenue", nbins=98)
fig.update_layout(title="Over 75% of orders are 5 or less", xaxis_title="Revenue", yaxis_title="Number of orders")
fig.show()

In [ ]:
print("Mean of orders of 15 or less:",orders[orders["revenue"] <= 15]["revenue"].mean())
print("Mean of orders of 20 or less:",orders[orders["revenue"] <= 20]["revenue"].mean())
print("Mean of orders of 25 or less:",orders[orders["revenue"] <= 25]["revenue"].mean())

The average purchase size could also be 3.39 to 3.79 depending on what you decide is an outlier unworthy of inclusion.

But have there been trends for *this* thing?

In [ ]:
grouped = orders.groupby("month", as_index=False).agg({"revenue": "mean"})
fig = px.line(grouped, x="month", y="revenue")
fig.update_layout(title="Average order size saw some weird swings in winter", xaxis_title="Month", yaxis_title="Average order size")
fig.show()

December and January were a bit weird, and though whenever anything store-related happens around then, it's tempting to blame Christmas, this dataset can't tell us what the deal truly was. Don't worry about June 2018, by the way; there's barely any data for it.

Maybe the deal was that we're including outliers? Here's another chart with orders of over 25 removed.

In [ ]:
not_big_orders = orders[orders["revenue"] <= 25]

grouped = not_big_orders.groupby("month", as_index=False).agg({"revenue": "mean"})
fig = px.line(grouped, x="month", y="revenue")
fig.update_layout(title="Average order size has generally declined", xaxis_title="Month", yaxis_title="Average order size")
fig.show()

It's totally different this way. After being 3.47 in June 2017, it shot up to 4.51 for the next month, then declined throughout the rest of the year and into January, going down to 3.56. By April it climbed back to around September levels, getting into the 3.90s, then May was really low again.

Alternate interpretation: July and August were strangely high, January wasn't good but that's normal for business, May was a blip, and things are fine. Maybe see what we had different in those two high months. And remember, this is per order, not total revenue. That downward trend is certainly worth thinking about, but it doesn't necessarily represent an overall loss in income.

#### Typical lifetime value and several cohort statistics

In [ ]:
grouped = user_info.groupby("uid", as_index=False).agg({"revenue": "sum"})
fig = px.histogram(grouped, x="revenue")
fig.update_layout(title="Most people don't ever buy much", xaxis_title="Revenue", yaxis_title="Users")
fig.show()

Well, that isn't very useful. Keeping it to those who made some sort of purchase, but a total of less than 25, we get this:

In [ ]:
fig = px.histogram(grouped[(grouped["revenue"] > 0) & (grouped["revenue"] < 25)], x="revenue", nbins=25)
fig.update_layout(title="Most people don't ever buy much", xaxis_title="Revenue", yaxis_title="Users")
fig.show()

But with all the outliers, because that is still real money that's coming in, what is the average lifetime value?

In [ ]:
ltv = grouped["revenue"].mean()
print(ltv)

And not counting the big ones:

In [ ]:
ltv_in = grouped[grouped["revenue"] <= 25]["revenue"].mean()
print(ltv_in)

Quite the difference. It'll be worth keeping the existence of outliers in mind.

Now time for a bunch of heatmaps. First up, customer counts.

In [ ]:
user_stuff = pd.DataFrame({"uid": user_info["uid"], "first_month": user_info["first_month"], "source_id": user_info["first_source"]})
orders_clutter = orders.merge(user_stuff, how="left", on="uid")
orders_clutter["lt_month"] = orders_clutter["month"] - orders_clutter["first_month"]
orders_clutter["lt_month"] = orders_clutter["lt_month"] / np.timedelta64(1, "M")
orders_clutter["lt_month"] = orders_clutter["lt_month"].round().astype("int")
customer_counts = orders_clutter.pivot_table(index="first_month", columns="lt_month", values="uid", aggfunc="nunique")

plt.figure(figsize=(15, 8))
plt.title("People who made an order")
sns.heatmap(customer_counts, annot=True, fmt=".0f", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"], cmap="YlGn", vmin=0, vmax=500)
plt.xlabel("Lifetime month")
plt.ylabel("First month")

Don't take the May cohort's month 1 seriously; again, June 2018 barely exists in this dataset.

In [ ]:
customer_counts_frac = customer_counts.divide(customer_counts.iloc[:, 0], axis=0)
plt.figure(figsize=(15, 8))
plt.title("Customer retention has been falling")
sns.heatmap(customer_counts_frac, annot=True, fmt=".3f", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"], cmap="YlGn", vmin=0, vmax=0.167)
plt.xlabel("Lifetime month")
plt.ylabel("First month")

Even among those who ever make an order, the later someone has visited for the first time, the less likely they are to make orders compared to their first couple months. Just like the visit retention rate, it's a worrying trend. Possibly the same trend.

But maybe those later joiners are rich or something. What about actual revenue?

In [ ]:
orders_clutter["lt_month"] = orders_clutter["month"] - orders_clutter["first_month"]
orders_clutter["lt_month"] = orders_clutter["lt_month"] / np.timedelta64(1, "M")
orders_clutter["lt_month"] = orders_clutter["lt_month"].round().astype("int")
oc_in = orders_clutter[orders_clutter["revenue"] <= 20]

cohort_revenue_sums = orders_clutter.groupby(["first_month", "lt_month"]).agg({"revenue": "sum"})
crs_pivot = cohort_revenue_sums.pivot_table(index="first_month", columns="lt_month", values="revenue", aggfunc="sum")

plt.figure(figsize=(15, 8))
plt.title("Revenue by month by cohort")
sns.heatmap(crs_pivot, annot=True, fmt=".0f", cmap="YlGn", vmin=0, vmax=5000, linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

In [ ]:
crs_in = oc_in.groupby(["first_month", "lt_month"]).agg({"revenue": "sum"})
crs_in_pivot = crs_in.pivot_table(index="first_month", columns="lt_month", values="revenue", aggfunc="sum")

plt.figure(figsize=(15, 8))
plt.title("Revenue by month by cohort (sans orders >20)")
sns.heatmap(crs_in_pivot, annot=True, fmt=".0f", cmap="YlGn", vmin=0, vmax=2500, linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

In [ ]:
crs_in_ptfrac = crs_in_pivot.divide(crs_in_pivot.iloc[:, 0], axis=0)
plt.figure(figsize=(15, 8))
plt.title("Revenue (sans orders >20) has also been dropping off more quickly")
sns.heatmap(crs_in_ptfrac, annot=True, fmt=".3f", cmap="YlGn", vmin=0, vmax=0.25, linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

Still a decline.

In [ ]:
revenue_per_customer_pt = crs_pivot / customer_counts
plt.figure(figsize=(15, 8))
plt.title("Average spending per customer")
sns.heatmap(revenue_per_customer_pt, annot=True, fmt=".2f", cmap="YlGn", vmin=0, vmax=20, linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

Revenue per user is lowest in a cohort's first month, which makes sense, since you'll have a lot of people who check out the site, decide it isn't what they need, and not do anything as far as money goes. People who come back will of course have a higher fraction of spenders than people coming for the first time.

For some reason, users who joined June 2017 and December 2017 bring quite a bit more income on average than other cohorts. We already like June 2017, but though December is no exception to the trend of declining retention, among those who do keep visiting, there's a lot of revenue. Or at least there is from someone....

In [ ]:
rpc_in_pt = crs_in_pivot / customer_counts
plt.figure(figsize=(15, 8))
plt.title("Cohort revenue averages per customer (sans orders >20)")
sns.heatmap(rpc_in_pt, annot=True, fmt=".2f", cmap="YlGn", vmin=0, vmax=10, linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

That December cohort's big numbers are from big orders rather than popularity, as we'll see later as well. But hey, it's been consistent.

Here's another way to look at things, the LTV per customer. The cohort's revenue accumulates rather than being only from the specific month, and everyone who's ever been a customer is counted in the averages even if they never show up again. To deal with outliers, if a user spent more than 25 in a month, it's counted as 25.

In [ ]:
oc_gr = orders_clutter.groupby(["month", "uid", "first_month", "lt_month"], as_index=False).agg({"revenue": "sum"})
oc_gr_capped = oc_gr
oc_gr_capped["revenue"] = oc_gr["revenue"].where(oc_gr["revenue"] <= 25, 25)

ocgrc_pt = oc_gr_capped.pivot_table(index="first_month", columns="lt_month", values="revenue", aggfunc="sum")
ocgrc_pt_cum = oc_gr_capped.pivot_table(index="first_month", columns="lt_month", values="revenue", aggfunc="sum").cumsum(axis=1)

customers_all_time = customer_counts.copy()
visitors_all_time = retention_pivot.copy()
for i in range(12):
    customers_all_time.iloc[i, :] = customers_all_time.iloc[i, 0]
    visitors_all_time.iloc[i, :] = visitors_all_time.iloc[i, 0]
customer_ltv_cap_pt = ocgrc_pt_cum / customers_all_time
visitor_ltv_cap_pt = ocgrc_pt_cum / visitors_all_time

plt.figure(figsize=(15, 8))
plt.title("LTV per customer (user-months capped at 25)")
sns.heatmap(customer_ltv_cap_pt, annot=True, fmt=".2f", cmap="Blues", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

In [ ]:
plt.figure(figsize=(15, 8))
plt.title("LTV per visitor (user-months capped at 25)")
sns.heatmap(visitor_ltv_cap_pt, annot=True, fmt=".2f", cmap="Blues", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

We see a smooth decline. Not fun.

There isn't a great segue, but here are some order numbers:

In [ ]:
cohort_orders = orders_clutter.pivot_table(index="first_month", columns="lt_month", values="buy_ts", aggfunc="count")
cohort_orders_per_customer = cohort_orders / customer_counts

plt.figure(figsize=(15, 8))
plt.title("Orders")
sns.heatmap(cohort_orders, annot=True, fmt=".0f", cmap="YlGn", vmin=0, vmax=720, linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

In [ ]:
plt.figure(figsize=(15, 8))
plt.title("Orders per customer")
sns.heatmap(cohort_orders_per_customer, annot=True, fmt=".2f", cmap="YlGn", vmin=1, vmax=2.5, linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

In months where they make an order, customers of the June 2017 cohort make the most, and by quite a distance. Other than that, it's all quite flat.

In [ ]:
table = orders.groupby("month", as_index=False).agg({"buy_ts": "count", "uid": "nunique"})
table = table[table["month"] != "2018-06-01"]
table_in = oc_in.groupby("month", as_index=False).agg({"buy_ts": "count"})
table_in = table_in[table_in["month"] != "2018-06-01"]
fig = go.Figure(data=[
    go.Scatter(mode="lines", name="All orders", x=table["month"], y=table["buy_ts"])
])
fig.update_yaxes(range=[0, 6250])
fig.update_layout(title="How many orders we get has seen some big swings", xaxis_title="Month", yaxis_title="Orders")
fig.show()

The first three months here are so low compared to the rest, then there's a huge rise. October to March is pretty much a plateau – you'd expect December to be high and January a bit lower – then April saw a 39% decline from the previous month. May is better, but not like where we were.

In [ ]:
fig = go.Figure(data=[
    go.Scatter(mode="lines", name="Orders per customer", x=table["month"], y=table["buy_ts"]/table["uid"])
])
fig.update_layout(title="Orders per customer is too spiky to see a trend", xaxis_title="Month", yaxis_title="Orders per customer")
fig.show()

It's much closer to the first month of a cohort than anything else. Again, a good majority of users have a tiny number of visits, so those in their first month will dominate the numbers.

In [ ]:
avg_order_revenue = orders_clutter.pivot_table(index="first_month", columns="lt_month", values="revenue", aggfunc="mean")
plt.figure(figsize=(15, 8))
plt.title("Average order size")
sns.heatmap(avg_order_revenue, annot=True, fmt=".2f", cmap="YlGn", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

In [ ]:
aor_in = oc_in.pivot_table(index="first_month", columns="lt_month", values="revenue", aggfunc="mean")
plt.figure(figsize=(15, 8))
plt.title("Average order size (sans orders >20)")
sns.heatmap(aor_in, annot=True, fmt=".2f", cmap="YlGn", linewidths=1, linecolor="#808080", yticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Lifetime month")
plt.ylabel("First month")

The first three cohorts make bigger orders. There's a noticeable step down.

In [ ]:
avg_order_size_in = oc_in.groupby("first_month", as_index=False).agg({"revenue": "mean"})
fig = px.line(avg_order_size_in, x="first_month", y="revenue")
fig.update_yaxes(range=[0, 4.25])
fig.update_layout(title="Post-August, typical orders are 14% smaller", xaxis_title="First month", yaxis_title="Average order size (sans orders >20)")
fig.show()

### Marketing
How much money was spent over the twelve months here, total and by source and month? How much did customer acquisition cost for each source, and what was the ROI?

#### How much was spent on marketing?
Well, the total is pretty easy:

In [ ]:
costs["costs"].sum()

By source:

In [ ]:
grouped = costs.groupby("source_id", as_index=False).agg({"costs": "sum"})
grouped["source_id"] = grouped["source_id"]
fig = px.bar(grouped, x="source_id", y="costs")
fig.update_layout(title="Source 3 had by far the most funding. Medium tier is 2, 4, and 5", xaxis_title="Source", yaxis_title="Expenditure", xaxis={"categoryorder": "total descending"})
fig.update_xaxes(type="category")
fig.show()

By month:

In [ ]:
costs["month"] = costs["dt"].astype("datetime64[M]")
grouped_m = costs.groupby("month", as_index=False).agg({"costs": "sum"})
fig = px.line(grouped_m, x="month", y="costs")
fig.update_yaxes(range=[0, 40000])
fig.update_layout(title="Marketing spend by month correlates well with visits", xaxis_title="Month", yaxis_title="Expenditure", xaxis_tickangle=-45)
fig.show()

By source and month:

In [ ]:
costs_grouped = costs.groupby(["source_id", "month"], as_index=False).agg({"costs": "sum"})
display(costs_grouped.pivot_table(index="source_id", columns="month", values="costs", aggfunc="sum", margins=True, margins_name="Total"))
fig = px.line(costs_grouped, x="month", y="costs", color="source_id")
fig.update_layout(title="Sources' changes in budget by month were pretty uniform", xaxis_title="Month", yaxis_title="Budget", legend_title_text="Source ID")
fig.show()

In [ ]:
fig = px.area(costs_grouped, x="month", y="costs", line_group="source_id", color="source_id")
fig.update_layout(title="Another view of how big source 3 and small 9 and 10 are", xaxis_title="Month", yaxis_title="Costs", legend_title_text="Source ID")
fig.show()

Now, that seeming correlation of marketing budget and visit number is interesting.... It seems like it should be a scatterplot.

In [ ]:
grouped_v = visits.groupby("month", as_index=False).agg({"device": "count"})
grouped_v = grouped_v.merge(grouped_m, how="left", on="month")
fig = px.scatter(grouped_v, x="costs", y="device", width=660, height=500)
fig.update_layout(title="Advertising: it really works", xaxis_title="Marketing budget", yaxis_title="Visits")
fig.show()

Look at how diagonal that is.

There isn't a notable difference between the devices here either, by the way. Here it is separated by them, with touch visits multiplied by the ratio of desktop to touch visits for better comparison.

In [ ]:
grouped_vd = visits_d.groupby("month", as_index=False).agg({"device": "count"})
grouped_vt = visits_t.groupby("month", as_index=False).agg({"device": "count"})
visits_by_m_adj = pd.DataFrame({"month": grouped_m["month"], "costs": grouped_m["costs"], "desktop": grouped_vd["device"], "touch": grouped_vt["device"]})
visits_by_m_adj["touch"] = visits_by_m_adj["touch"] * visits_by_m_adj["desktop"].sum() / visits_by_m_adj["touch"].sum()

fig = px.scatter(visits_by_m_adj, x="costs", y=["desktop", "touch"], width=660, height=500)
fig.update_layout(title="Advertising: it really works", xaxis_title="Marketing budget", yaxis_title="Visits (adjusted)", legend_title_text="Device")
item_labels = ["Desktop", "Touch"]
for i, label in enumerate(item_labels):
    fig.data[i].name = label
fig.show()

#### Customer acquisition costs
First of all, are some sources a lot better at attracting visitors than others?

In [ ]:
grouped_c = costs.groupby("source_id", as_index=False).agg({"costs": "sum"})
grouped_v = visits.groupby("source_id", as_index=False).agg({"device": "count"})
grouped_v = grouped_v.merge(grouped_c, how="left", on="source_id")
grouped_v = grouped_v.fillna(0)
grouped_v["source_id"] = grouped_v["source_id"].astype("str")
fig = px.scatter(grouped_v, x="costs", y="device", color="source_id", width=660, height=500)
fig.update_layout(xaxis_title="Marketing budget", yaxis_title="Visits")
fig.update_layout(legend={"yanchor": "top", "xanchor": "left", "x": 0.01, "y": 0.99, "title": "Source ID"})
fig.update_xaxes(dtick=20000)
fig.show()

I'm not saying source 3 is bad, but it at least found saturation. And as shown previously, it didn't have a massive spike in one month to screw with the total, and neither did any of the others. But what about a big spike in visits?

In [ ]:
source_visits_by_month = visits.groupby(["month", "source_id"], as_index=False).agg({"start_ts": "count"})
source_visits_by_month = source_visits_by_month.rename(columns={"start_ts": "visits"})
fig = px.line(source_visits_by_month, x="month", y="visits", color="source_id", category_orders={"source_id": [1, 2, 3, 4, 5, 6, 7, 9, 10]})
fig.update_layout(title="Visits from each source by month rose and fell similarly", xaxis_title="Month", yaxis_title="Visits", legend_title_text="Source ID")
fig.show()

Somtimes you'll see a little spike or flat line in one source where others have something flatter or steeper, but everything has the same general pattern. There isn't really anything to act on or to suggest the other data by month needs to be adjusted somehow.

Now for some *customer* acquisition cost calculations, not caring about visitors who never converted. First of all, plainly by month:

In [ ]:
cgroup = costs.groupby("month", as_index=False).agg({"costs": "sum"})
ocgroup = orders_clutter.groupby("first_month", as_index=False).agg({"uid": "nunique"})
ocgroup = ocgroup.rename(columns={"first_month": "month"})
cocmerge = cgroup.merge(ocgroup, how="inner", on="month")
cocmerge["cac"] = cocmerge["costs"] / cocmerge["uid"]

In [ ]:
first_order_month = orders.groupby("uid")["buy_ts"].min()
first_order_month.name = "first_order_month"
user_info = user_info.join(first_order_month, on="uid")
orders_clutter = orders_clutter.join(first_order_month, on="uid")
user_info["first_order_month"] = user_info["first_order_month"].astype("datetime64[M]")
orders_clutter["first_order_month"] = orders_clutter["first_order_month"].astype("datetime64[M]")

costs_by_month = costs.groupby("month").agg({"costs": "sum"})
first_o_month_users = orders_clutter.groupby("first_order_month").agg({"uid": "nunique"})
first_o_month_users.index.name = "month"
cacmerge = costs_by_month.merge(first_o_month_users, how="inner", on="month")
cacmerge["cac"] = cacmerge["costs"] / cacmerge["uid"]
step = pd.DataFrame(data={"month": cacmerge.index, "cac": cacmerge["cac"]})

fig = px.line(step, x="month", y="cac")
fig.update_layout(title="CAC generally hovers around 8.5 to 10", xaxis_title="Month", yaxis_title="Customer acquisition cost")
fig.show()

CAC was rather high in August 2017 and low in May 2018, but the former didn't foretell a trend and the latter probably doesn't either.

Looking at the individual sources might be more helpful.

In [ ]:
cac_two_grouped = user_info.groupby(["first_source", "first_order_day"]).agg({"uid": "count"})
cac_two_merged = cac_two_grouped.merge(costs, how="left", left_on=["first_source", "first_order_day"], right_on=["source_id", "dt"])

cac_two_merged = cac_two_merged.dropna()
cac_two_merged["source_id"] = cac_two_merged["source_id"].astype("int")
cac_two_merged = cac_two_merged.rename(columns={"uid": "users_drawn"})

ctwm_grouped = cac_two_merged.groupby(["source_id", "month"], as_index=False).agg({"costs": "sum", "users_drawn": "sum"})
ctwm_grouped["cac"] = ctwm_grouped["costs"] / ctwm_grouped["users_drawn"]

fig = px.line(ctwm_grouped, x="month", y="cac", color="source_id")
fig.update_layout(title="Source 2 has not been efficient at drawing purchasers", xaxis_title="Month", yaxis_title="Advertising spend per purchaser", legend_title_text="Source ID")
fig.show()

Sources 4, 9, and 10 really "know" how to attract people who convert, 1 and 5 are the middling ones, and 2 and 3 are pretty sad.

Some of them have not exactly trends but steps that raise a question or two. Source 4 got better starting October, what if anything was changed then? Similarly, source 5 got worse in November. Were we doing anything different with source 2 in May? If this hasn't been looked into already, there might be some things we can learn there, or at least pointers to experiments.

Have it in heatmap form too:

In [ ]:
ctwmgpt = ctwm_grouped.pivot_table(index="source_id", columns="month", values="cac", aggfunc="sum")
plt.figure(figsize=(15, 8))
plt.title("CAC by source by month")
sns.heatmap(ctwmgpt, annot=True, fmt=".2f", linewidths=1, linecolor="#808080", xticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"], cmap="Reds", vmin=0, vmax=20)
plt.xlabel("Month")
plt.ylabel("Source")

And here are the sources overall:

In [ ]:
pdbs = orders_clutter.groupby("source_id", as_index=False).agg({"uid": "nunique"})
pdbs = pdbs.rename(columns={"uid": "users_drawn"})
pdbs = pdbs[pdbs["source_id"] != 7]
total_costs = costs.groupby("source_id", as_index=False).agg({"costs": "sum"})
total_costs = total_costs.merge(pdbs, how="inner", on="source_id")
total_costs["source_id"] = total_costs["source_id"].astype("str")
total_costs["cac"] = total_costs["costs"] / total_costs["users_drawn"]

#ctwm_g_squ = ctwm_grouped.groupby("source_id", as_index=False).agg({"costs": "sum", "users_drawn": "sum"})
#ctwm_g_squ["cac"] = ctwm_g_squ["costs"] / ctwm_g_squ["users_drawn"]
#ctwm_g_squ["source_id"] = ctwm_g_squ["source_id"].astype("str")

fig = px.bar(total_costs, x="source_id", y="cac")
fig.update_layout(title="Sources' 2 and 3's CACs are 70% higher than anything else", xaxis_title="Source ID", yaxis_title="Customer acquisition cost")
fig.show()

#### ROI calculations
By source across the whole 12 months, we have this:

In [ ]:
purchaser_info = user_info[user_info["revenue"] > 0]
grouped_c = costs.groupby("source_id", as_index=False).agg({"costs": "sum"})

grouped = user_info.groupby("first_source", as_index=False).agg({"revenue": "sum"})
grouped_in = user_info[user_info["revenue"] < 25]
grouped_in = grouped_in.groupby("first_source", as_index=False).agg({"revenue": "sum"})
grouped = grouped.merge(grouped_in, on="first_source", how="left")
grouped = grouped.rename(columns={"first_source": "source_id", "revenue_x": "revenue", "revenue_y": "revenue_<25"})

grouped_c = grouped_c.merge(grouped, on="source_id", how="left")
grouped_c["roi"] = grouped_c["revenue"] / grouped_c["costs"]
grouped_c["roi_<25"] = grouped_c["revenue_<25"] / grouped_c["costs"]
grouped_c["source_id"] = grouped_c["source_id"].astype("str")

att_iii = purchaser_info.groupby(["first_month", "first_source"], as_index=False).agg({"uid": "count"})
att_iii = att_iii.rename(columns={"first_month": "month", "first_source": "source_id"})

costs_by_month = costs.groupby(["month", "source_id"], as_index=False).agg({"costs": "sum"})

att_iii = att_iii.merge(costs_by_month, how="left", on=["month", "source_id"])
att_iii = att_iii[att_iii["source_id"] != 7]
att_iii["cac"] = att_iii["costs"] / att_iii["uid"]

orders_clutter = orders_clutter.rename(columns={"first_source": "source_id"})
revenue_months = orders_clutter.groupby(["month", "source_id"], as_index=False).agg({"revenue": "sum"})
revenue_months = revenue_months[revenue_months["source_id"] != 7]

att_iii = att_iii.merge(revenue_months, how="left", on=["month", "source_id"])
att_iii["roi"] = att_iii["revenue"] / att_iii["costs"]

source_overall = att_iii.groupby("source_id").agg({"costs": "sum", "revenue": "sum"})
source_overall["roi"] = source_overall["revenue"] / source_overall["costs"]

fig = go.Figure(data=[
    go.Bar(name="All users", x=grouped_c["source_id"], y=grouped_c["roi"]-1),
    go.Bar(name="Users LTV <25", x=grouped_c["source_id"], y=grouped_c["roi_<25"]-1)
])
fig.update_layout(title="Source 3 has been the main thing bringing us down", xaxis_title="Source ID", yaxis_title="ROI")
fig.show()

Most but not all of the sources are just about worth it, and that depends on the rare massive orders coming around, which probably isn't something to rely on. Source 3 in particular has been terrible, but again, it may just be overfed.

And here it is by month overall:

In [ ]:
orders_by_m = orders.groupby("month", as_index=False).agg({"uid": "count"})
users_by_m = visits.groupby("month", as_index=False).agg({"uid": "nunique"})
buyers_by_m = orders.groupby("month", as_index=False).agg({"uid": "nunique"})
revenue_by_m = orders.groupby("month", as_index=False).agg({"revenue": "sum"})
costs_by_m = costs.groupby("month", as_index=False).agg({"costs": "sum"})

m_aggs = pd.DataFrame(data={"month": orders_by_m["month"], "users": users_by_m["uid"], "buyers": buyers_by_m["uid"], "orders": orders_by_m["uid"], "costs": costs_by_m["costs"], "revenue": revenue_by_m["revenue"]})
m_aggs = m_aggs[m_aggs["month"] != "2018-06-01"]
m_aggs["users"] = m_aggs["users"].astype("int")
m_aggs["roi"] = (m_aggs["revenue"] / m_aggs["costs"]) - 1

m_aggs["costs_cum"] = m_aggs["costs"].cumsum()
m_aggs["revenue_cum"] = m_aggs["revenue"].cumsum()
m_aggs["roi_cum"] = (m_aggs["revenue_cum"] / m_aggs["costs_cum"]) - 1

fig = go.Figure(data=[
    go.Scatter(mode="lines", name="Individual months", x=m_aggs["month"], y=m_aggs["roi"]),
    go.Scatter(mode="lines", name="Cumulative", x=m_aggs["month"], y=m_aggs["roi_cum"])
])
fig.update_layout(title="ROI has been increasing but remains negative", xaxis_title="Month", yaxis_title="ROI")
fig.show()

The ROI's had some big swings, but the trend is positive. It hasn't ever been above zero, though.

By source and month:

In [ ]:
source_revenues = orders_clutter.groupby(["source_id", "month"], as_index=False).agg({"revenue": "sum"})
costs_by_month = costs.groupby(["source_id", "month"], as_index=False).agg({"costs": "sum"})
source_rois = source_revenues.merge(costs_by_month, how="inner", on=["source_id", "month"])
source_rois["roi"] = (source_rois["revenue"] - source_rois["costs"]) / source_rois["costs"]

source_roi_pivot = source_rois.pivot_table(index="source_id", columns="month", values="roi", aggfunc="sum")
plt.figure(figsize=(15, 8))
plt.title("Sources' ROIs on individual months")
sns.heatmap(source_roi_pivot, annot=True, fmt=".2f", linewidths=1, linecolor="#808080", cmap="RdYlGn", center=0, vmin=-1, vmax=1, xticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Month")
plt.ylabel("Source ID")

In [ ]:
source_rois["revenue_cum"] = source_rois.groupby("source_id")["revenue"].cumsum()
source_rois["costs_cum"] = source_rois.groupby("source_id")["costs"].cumsum()
source_rois["roi_cum"] = (source_rois["revenue_cum"] / source_rois["costs_cum"]) - 1

source_roi_cum_pt = source_rois.pivot_table(index="source_id", columns="month", values="roi_cum", aggfunc="sum")
plt.figure(figsize=(15, 8))
plt.title("Running source ROIs")
sns.heatmap(source_roi_cum_pt, annot=True, fmt=".2f", linewidths=1, linecolor="#808080", cmap="RdYlGn", center=0, vmin=-1, vmax=1, xticklabels=["2017-06", "2017-07", "2017-08", "2017-09", "2017-10", "2017-11", "2017-12", "2018-01", "2018-02", "2018-03", "2018-04", "2018-05"])
plt.xlabel("Month")
plt.ylabel("Source ID")

Source 1 has been consistently profitable, 2 got better in the last four months to the point of turning a profit over the time being measured, 3 has always been bad, 4 has got better over time but has never been great, 5 had one massive month but has almost always been bad, 9 is pretty unpredictable month to month but generally good, and 10 has always been bad until the last three months.

## Recommendations
### First of all, this isn't a situation of no hope
Despite the overall stats, the whole general idea of advertising for the site is not necessarily a bad one. If it really is true about source 3 that it was totally saturated, that alone makes things look much better: if you make its costs for every month equal to the average of those for 2, 4, and 5 but keep the same visitors, seven of the last eight months are profitable, though still relying on the big purchases for that to be the case.

### What to focus our creativity on
The plot of visits and funding by source is pretty linear in general, suggesting that the marketing gets people to come to the site, that isn't the problem. What we need to come up with is how to get a good fraction of people to buy things once they're there. Getting people to stay after they've come for the first time also has a lot of room for improvement, and combined with a solution to the low conversion, that should change that not many people buy anything after their first few days.

### The sources
* Source 1 relies on bigger users a lot, but it's proven consistent, so there's no reason not to keep it.
* Source 2 also has a huge reliance on big spenders, as shown by its bad CAC and its ROI when only counting "normal" spenders. It's got a lot better in the last few months, though. It might be good to keep it where it is for now and see if it keeps it up but also not be afraid to drop it.
* Source 3 has found a limit and should be brought back to the moderate level some of the others were at, around 50k a year, for proper evaluation.
* Even though it mostly hasn't been profitable, source 4 knows how to bring people in, and if we do figure out the above problems, it could end up being the best one. Maybe suspend it until we're better at converting, but certainly don't forget about it.
* Source 5 is iffy on visits, okay but not great on users, and middling on actual ROI (and look how much of it was from a single month). It's closer to bad than good, and if we improve the overall situation, it might turn out to be okay, but if we're looking to drop something, it won't be a terrible choice. Maybe drop it for now and bring it back for another chance if or when things improve in general.
* Source 9 has encouraging stats and should be given a proper chance. Like source 3, it should get a medium amount of funding.
* Over the year this analysis is about, source 10 seems good at attracting a lot of users and even users who convert, but apparently not ones willing to spend much. In the past three months it's been noticeably better, but that's true for most of the sources. I wouldn't abandon hope, but maybe keep it small for now.


Considering how closely visits and advertising are correlated, steadily increasing the budget in general until the returns diminish will be a good idea, *after* we find profitability with the whole endeavour in the first place.